In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from evaluation_metric import amex_metric

import warnings
warnings.filterwarnings("ignore")

In [ ]:
p_train = pd.read_pickle('Output/p_train_xgb_b2.pkl')
train_labels = pd.read_pickle('Data/train_labels.pkl').loc[p_train.index]
p_train.shape, train_labels.shape

In [ ]:
df_study = pd.DataFrame(p_train.mean(axis=1), columns=['mean'])
df_study['std'] = p_train.std(axis=1)
df_study['median'] = p_train.median(axis=1)

In [ ]:
def top_percent_captured(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        pct_cutoff = int(0.36 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= pct_cutoff]
        print((df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum())
        return df_cutoff
        
df_cutoff = top_percent_captured(train_labels, df_study.rename({'mean': 'prediction'}, axis=1))

In [ ]:
df_cutoff.target.value_counts()

In [ ]:
amex_metric(train_labels.loc[df_cutoff.index], df_cutoff.rename({'mean': 'prediction'}, axis=1)['prediction'].to_frame())

### Saving Indices

In [ ]:
pd.DataFrame(df_cutoff.index.to_list()).to_csv('Data/top_indx.csv')

### Seperate bad from good

In [ ]:
train_data = pd.read_pickle('Data/train_data.pkl').loc[df_cutoff.index]
train_data.shape

In [ ]:
train_data